In [1]:
import warnings
from warnings import simplefilter
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=PendingDeprecationWarning)

In [2]:
import os
import rpy2
import logging
import anndata2ri
import scanpy as sc
import decoupler as dc
import matplotlib.pyplot as plt
import rpy2.robjects as robjects
from itertools import chain
from rpy2.robjects import pandas2ri
from functions import helper_functions

In [3]:
# # Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()

-----
anndata     0.11.1
scanpy      1.9.3
-----
PIL                         9.5.0
anndata2ri                  1.1
asttokens                   NA
backcall                    0.2.0
cffi                        1.15.1
comm                        0.1.3
cycler                      0.10.0
cython_runtime              NA
dateutil                    2.8.2
debugpy                     1.6.7
decorator                   5.1.1
decoupler                   1.4.0
dot_parser                  NA
exceptiongroup              1.1.1
executing                   1.2.0
functions                   NA
google                      NA
gsva_prep                   NA
h5py                        3.9.0
igraph                      0.10.4
ipykernel                   6.23.2
ipywidgets                  8.0.6
jedi                        0.18.2
jinja2                      3.1.2
joblib                      1.2.0
kiwisolver                  1.4.4
leidenalg                   0.9.1
llvmlite                    0.39.1
louvain      

In [4]:
%%R

suppressPackageStartupMessages({
    library(Matrix)
    library(viridis)
    library(harmony)
    library(ggpubr)
    library(tictoc)
    library(RColorBrewer)
    library(Hmisc)
    library(corrplot)
    library(grid)
    library(gridExtra)
    library(igraph)
    library(ggrepel)
    library(readxl)
    library(conflicted)
    library(plyr)
    library(dplyr)
    library(parallel)
    library(stringr)
    library(BiocParallel)

    # single-cell analysis package
    library(Seurat)
    library(zellkonverter)   
    library(SingleCellExperiment)
    library(tidyr)
    library(readxl)
    library(GSA)
    library(limma)

    # plotting and data science packages
    library(tidyverse)
    library(cowplot)
    library(patchwork)
    library(ggplot2)

    # co-expression network analysis packages:
    library(WGCNA)
    library(hdWGCNA)

    # gene enrichment packages
    library(enrichR)
    library(GeneOverlap)
    library(GSEABase)
    library(GSVA) 

    # cell-cell communication
    library(nichenetr)

# needs to be run every time you start R and want to use %>%
})

# using the cowplot theme for ggplot
theme_set(theme_cowplot())

# set random seed for reproducibility
set.seed(12345)

# optionally enable multithreading
enableWGCNAThreads(nThreads = 40)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    Allowing parallel execution with up to 40 working processes.


In addition: Warning messages:
1: replacing previous import ‘GenomicRanges::intersect’ by ‘SeuratObject::intersect’ when loading ‘hdWGCNA’ 
2: replacing previous import ‘GenomicRanges::union’ by ‘dplyr::union’ when loading ‘hdWGCNA’ 
3: replacing previous import ‘GenomicRanges::setdiff’ by ‘dplyr::setdiff’ when loading ‘hdWGCNA’ 
4: replacing previous import ‘dplyr::as_data_frame’ by ‘igraph::as_data_frame’ when loading ‘hdWGCNA’ 
5: replacing previous import ‘Seurat::components’ by ‘igraph::components’ when loading ‘hdWGCNA’ 
6: replacing previous import ‘dplyr::groups’ by ‘igraph::groups’ when loading ‘hdWGCNA’ 
7: replacing previous import ‘dplyr::union’ by ‘igraph::union’ when loading ‘hdWGCNA’ 
8: replacing previous import ‘GenomicRanges::subtract’ by ‘magrittr::subtract’ when loading ‘hdWGCNA’ 
9: replacing previous import ‘Matrix::as.matrix’ by ‘proxy::as.matrix’ when loading ‘hdWGCNA’ 
10: replacing previous import ‘igraph::groups’ by ‘tidygraph::groups’ when loading ‘hdWGCNA’ 

# **Systematic differential analysis of pathway activity**

Pathway activity scores were computed in accordance with the overall protocol outlined in [**Joel W. Blanchard et. al.**](https://doi.org/10.1038/s41586-022-05439-w).

- Aggregate gene expression profiles into `pseudo-cells` or `pseudo-bulks` depending on the desired strategy in **[Gazestani. et. al. 2023](https://www.sciencedirect.com/science/article/pii/S0092867423008590?via%3Dihub)** (`pseudo-cells`), [**Joel W. Blanchard et. al.**](https://doi.org/10.1038/s41586-022-05439-w) (`pseudo-bulks`), or [**Morabito et al.**](https://www.sciencedirect.com/science/article/pii/S2667237523001273?via%3Dihub) (`meta-cells`)
 

- Use `PAGODA2`, `GSVA`, or `AUCell`, to obtain pathway activity scores for pathways (`GO Ontology Biological Processes`) obtained from [**Maayan Lab**](https://maayanlab.cloud/Enrichr/#libraries), [**MSigDB**](https://www.gsea-msigdb.org/gsea/msigdb/human/collections.jsp#C5) or custom gene sets.

- Perform differential pathway activity analysis using limma


## **Data Prep Parameters**

- `test_names`: List of the different test names of interest.

- `save_prefix`: Preferred prefix for saving critical files. Ideally chosen to be in the format `{source name}_{brain region}`. e.g `mathys_pfc`

- `subject_id`: Column name for Subject/Patient ID in both metadata and `.obs`

In [5]:
save_prefix = 'seaad_mtg'                           # this takes the format '{StudyName}_{ThreeLetterAccronymForBrainRegion}'
cell_group = 'excitatory'
cell_group_number = 7
subject_id = 'Donor ID' 
cell_type_column = 'Subclass'                       # 'Supertype (non-expanded)', 'Subclass'
pseudobulk_level = 'Supertype'                      # Level at which cells are aggregated into pseudobulks,
                                                    # 'Class', 'Subclass', or 'Supertype' 
factor = 'pathology.group'                          # pathology.group # Continuous Pseudoprogression Score
factor = factor.replace(" ", "").replace("-", "")
test_names = ['late_vs_early']                      # test categories
region_name = save_prefix.split('_')[-1].upper()

data_dir = f'/media/tadeoye/Volume1/data/seq/SEA-AD/{region_name}/RNAseq'
save_dir = f'../results/'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

subclass = {
    'excitatory': ['L5 IT', 'L2/3 IT', 'L4 IT', 'L6 IT', 'L6 IT Car3', 'L5/6 NP', 'L6b', 'L6 CT', 'L5 ET'],
    'inhibitory': ['Pvalb', 'Sst', 'Lamp5 Lhx6', 'Vip', 'Lamp5', 'Sncg', 'Chandelier', 'Sst Chodl', 'Pax6'],
    'astrocyte': ['Astrocyte'],
    'microglia': ['Microglia-PVM'],
    'opc': ['OPC'],
    'oligodendrocyte': ['Oligodendrocyte'],
    'endothelial': ['Endothelial'],
    'vlmc': ['VLMC'],
    }

if cell_group=="all":
    cell_supertypes = list(chain(*list(subclass.values()))) 
else:
    cell_supertypes = [subclass[cell_group][cell_group_number]]


## **Load Standardized Data**

We have previously standardized and filtered the `adata` obj for each cell sypertype (`00_Standardize_and_Split_RNAseq.ipynb`). We implement the comprehensive covariate standardization and categorization employed **[here](https://github.com/AllenInstitute/SEA-AD_2024/blob/main/Single%20nucleus%20omics/04_Differential%20expression%20analysis/00_Split%20AnnData%20for%20nebula.ipynb)** to ensure robust statistical comparisons. 

Continuous variables were normalized to a [0,1] interval using min-max scaling, including `post-mortem interval (PMI)` and `the number of genes detected per cell`. 

`Age at death` was discretized into five equal-width bins and subsequently normalized to account for non-linear age effects.

Categorical variables underwent systematic preprocessing to ensure consistent encoding. `Sex` was binary-encoded as F/M, while `APOE4` carrier status was derived from genotype information and encoded as Y/N. Race information, specifically White identification, was encoded as Checked/Unchecked. Technical covariates, including sequencing method and donor ID, were retained with cleaned category levels. 

For quality control metrics, we normalized the `number of unique molecular identifiers (UMIs)` and `genes detected per cell`,to enabling direct comparisons across samples.

Here, we load the standardized `h5ad` file. 

In [6]:
if cell_group == "all":
    print(f'loading data for all {cell_type_column} cell types...')
    adata_annot = sc.read_h5ad(data_dir+f'counts/anndata/all_subclass_standardized_anndata.h5ad')
    adata_annot.X = adata_annot.X.toarray()
    adata_annot.layers['counts'] = adata_annot.layers['UMIs'].toarray()
    del adata_annot.layers['UMIs']
else:
    print(f'loading data for {cell_supertypes[0]}...')
    adata_annot = sc.read_h5ad(os.path.join(
        data_dir,
        'counts',
        cell_group,
        helper_functions.clean_strings(cell_supertypes[0], preserve_case=True),
        f'{helper_functions.clean_strings(cell_supertypes[0], preserve_case=True)}_standardized_anndata.h5ad'
        )
    )

adata_annot

loading data for Pvalb...


AnnData object with n_obs × n_vars = 76958 × 36601
    obs: 'method', 'Subclass', 'Supertype (non-expanded)', 'Supertype', 'continuous_pseudo_progression_score', 'age_at_death_binned_codes', 'sex', 'race_choice_white', 'genes_detected', 'donor_id', 'number_of_umis', 'pmi', 'apoe4_status'
    var: 'gene_ids'
    uns: 'APOE4 Status_colors', 'Braak_colors', 'CERAD score_colors', 'Cognitive Status_colors', 'Great Apes Metadata', 'Highest Lewy Body Disease_colors', 'LATE_colors', 'Overall AD neuropathological Change_colors', 'Sex_colors', 'Subclass_colors', 'Supertype_colors', 'Thal_colors', 'UW Clinical Metadata', 'X_normalization', 'batch_condition', 'default_embedding', 'neighbors', 'title', 'umap'
    obsm: 'X_scVI', 'X_umap'
    layers: 'UMIs'
    obsp: 'connectivities', 'distances'

# **Parameters for Testing Differential Pathway Activity**

The primary differential pathway activity analaysis:

- `covariates`: Specifies variables to account for in the analysis:
    - Must include `Continuous Pseudo-progression Score` which will later on be matched to`pathology.group` as the primary variable of interest.

    - Can include additional confounding variables like:
        - Demographic factors (e.g., `Sex`, `Age`)
        - Technical factors (e.g., `Sample Batch`)

    - If there are no additional confounders, set `covariates = ['Continuous Pseudo-progression Score']`.

    - For the best results, ensure:
        - continuous covariates are similarly scaled.
        - uniformity in categorical data. E.g., Avoid having 'M', 'Male', and 'male' in the same dataset.
        - regularly assess the impact of covariates by running the analyses with and without each covariate. Watch out for covariates unduly influencing results. Watch out for covariates unduly influencing results.

    - **`The available covariates in the standardized AnnData object are strictly limited to those established during the initial preprocessing step in 00_Standardize_and_Split_RNAseq.ipynb`**. To incorporate additional covariates, return to `00_Standardize_and_Split_RNAseq.ipynb` and add desired covariates during standardization then re-run the complete preprocessing pipeline.

- `random_effect`: Identifies the technical variable to be treated as random effects (in this case `subject_id` or `Sample_batch`). These effects are regressed out using `duplicatecorrection`. This would help account or technical variation not relevant to the biological question.

- `jack_knifing`: Boolean parameter controlling resampling analysis. When set to `True`, the script performs repeated analysis on data subsets to assess robustness of identified pathway activity patterns. This would help identify patterns that might be driven by outliers or specific samples.

In [7]:
######################### Differential Pathway Activity arguments #########################
jack_knifing = True                                                 # Boolean parameter controlling resampling analysis
random_effect = helper_functions.clean_strings([subject_id])[0]     # Technical covariate to be included as random effect not of interest (regressed out by duplicatecorrection)
subject_id = random_effect

covariates = [
    'Continuous Pseudo-progression Score',
    'Age at Death binned codes',
    'Sex',
    'PMI',
    'Genes detected',
    'Number of UMIs',
    'method',
    'Race (choice=White)',
    'APOE4_Status'
]

covariates = helper_functions.clean_strings(covariates, replace_hyphen=True)

numeric_covariates = helper_functions.clean_strings(['Age at Death binned codes',
                                            'Continuous Pseudo-progression Score', 
                                            'PMI',
                                            'Genes detected',
                                            'Number of UMIs'],
                                            replace_hyphen=True)

# confirm that all covariates and numeric covariates are in adata.obs

for covariate in covariates + numeric_covariates:
    assert covariate in adata_annot.obs.columns, f"Missing covariate: {covariate}"

# **Count Aggregation**

### **Count Aggregation Parameters**

Since we plan on using a pseudobulking strategy to aggregate single-cell data into more robust representative samples, we specify the following parameters:

- `count_agg_strategy`: Defines the pseudobulking strategy and includes options such as 
    - `standardbulk`: Uses `decoupler` to aggregate cells by summing raw counts for each gene across all cells that share the same sample ID and cell type.

    - `blanchardbulk`: Implements the pseudobulking strategy From [Blanchard et al. Nature. 2022](https://doi.org/10.1038/s41586-022-05439-w)

    - `network`, `smaller_network`, or `random`: A set of pseudocell strategies adopted from [Gazestani. et. al. Cell. 2023](https://www.sciencedirect.com/science/article/pii/S0092867423008590?via%3Dihub) to either randomly assign cells to pseudocells in each cell type from each individual (`random`); or create a similarity network between these cells and use that similarity network to group cells with highly similar expression patterns to each other (`network`) at varying sizes (`smaller network`).
    
    - `metacell`: Implements the hdWGCNA [Morabito et al. Cell Reports Methods. 2023](https://www.sciencedirect.com/science/article/pii/S2667237523001273?via%3Dihub) bootstrapped aggregation (bagging) strategy to reduce matrix sparsity. LIke [Gazestani. et. al. Cell. 2023](https://www.sciencedirect.com/science/article/pii/S0092867423008590?via%3Dihub) it uses k-nearest neighbors to identify transcriptionally similar cells in a reduced dimensional space.

- `reconstruct_embedding`: Controls whether to reconstruct the embedding based on the specified `count_agg_strategy`. Cellular embedding reconstruction is performed distinctly for `network`, `smaller network`, and `metacell` aggregation strategies. For `network` and `smaller network` strategies, final embedding is extracted from top principal components, derived from the normalized and scaled data. The `metacell` strategy incorporates additional `Harmony` correction to account for batch effects across subjects.

- `emedding_name`: This allows for selection of pre-computed dimensional reductions stored within the AnnData object. If `reconstruct_embedding = False`, it specifies the key corresponding to the desired embedding matrix in the `.obsm` attribute of the AnnData object. The specified key must exist within the `.obsm` dictionary and is analogous to the `reducedDimNames` nomenclature used for SIngleCellEXperiment objects.

- `gene_celltype_threshold`: Determines the number of cells a gene must be expressed in.

- `pathway_gene_threshold`: Determines the number of genes that must be present in a pathway.

- `min_size_limit`: Specifies the minimum acceptable size of the pseudobulk data, with pseudobulks below this cell threshold excluded from the analysis.

In [8]:
filter_genes = "TRUE"
count_agg_strategy = 'metacell'   # options 'network', 'random', 'pseudobulk', 'smaller_network', 'standardbulk', 'metacell', `blanchardbulk``
reconstruct_embedding = 'FALSE'
embedding_name = 'X_scVI'
gene_celltype_threshold = 0.10    # determines number of cells the gene must be expressed in
pathway_gene_threshold = 0.33     # determines number of genes that must be present in that pathway
min_size_limit = 30               # min_size_limit: the minimum acceptable size of the pseudobulk data.
                                  # min_size_limit: pseudobulks with cells less than this threshold are excluded from the analysis 

### **Send parameters to R interface with rpy2**

In [9]:
%%R -i adata_annot -i subject_id -i cell_type_column -i cell_group -i gene_celltype_threshold -i cell_supertypes -i pseudobulk_level -i test_names -i save_prefix -i save_dir -i data_dir -i subject_id -i covariates -i random_effect -i min_size_limit -i count_agg_strategy -i numeric_covariates -i reconstruct_embedding -i embedding_name

# adata_annot <- readH5AD(paste0(data_dir, "anndata/", save_prefix, "_standardized_anndata.h5ad"), use_hdf5=TRUE, reader='R')

names(assays(adata_annot)) = c("X", "counts")

print('loaded data into memory for recursive use')
print(adata_annot)

[1] "loaded data into memory for recursive use"
class: SingleCellExperiment 
dim: 36601 76958 
metadata(19): APOE4 Status_colors Braak_colors ... title umap
assays(2): X counts
rownames(36601): MIR1302-2HG FAM138A ... AC007325.4 AC007325.2
rowData names(1): gene_ids
colnames(76958): CCCTTAGCATTAAAGG-L8TX_210826_01_B06-1153814368
  TAAGCCATCGCATTAG-L8TX_210708_01_D09-1153814279 ...
  GAGTTACAGCGTGTCC-L8TX_210430_01_B04-1153814208
  TACACCCAGGTTCCGC-L8TX_210429_01_D03-1142430416
colData names(13): method Subclass ... pmi apoe4_status
reducedDimNames(2): X_scVI UMAP
mainExpName: NULL
altExpNames(0):


## **Standard Pseudobulk Approach**

To create pseudobulk data per donor and cell_type

### **Pseudo-bulk With Decoupler**

#### **Requied Parameters when using decoupler**


- `min_counts`: Filter to remove samples by a minimum number of summed counts in a sample-group pair when using standard pseudobulk method

- `min_count`: Minimum count requiered per gene for at least some samples.

- `min_total_count`: Minimum total count required per gene across all samples.

- `large_n`: Number of samples per group that is considered to be "large".
    
- `min_prop`: Minimum proportion of samples in the smallest group that express the gene.

In [10]:
# Get pseudo-bulk profile

min_counts = 1000       #    Filter to remove samples by a minimum number of summed counts in a sample-group pair when using standard pseudobulk method
min_count = 10          #    Minimum count requiered per gene for at least some samples.
min_total_count = 15    #    Minimum total count required per gene across all samples.
large_n = 1             #    Number of samples per group that is considered to be "large".
min_prop = 1            #    Minimum proportion of samples in the smallest group that express the gene.

if count_agg_strategy == 'standardbulk':
    
    pdata = dc.get_pseudobulk(
        adata_annot,
        sample_col=subject_id,
        groups_col=pseudobulk_level,
        # layer='counts',
        mode='sum',
        min_cells=min_size_limit, 
        min_counts=min_counts,
    )

    # Sample-cell-type QC visualization
    dc.plot_psbulk_samples(pdata, 
                           groupby=[subject_id, pseudobulk_level], 
                           figsize=(12, 6))
    
    summed_counts_per_celltype = {}
    expressed_genes_per_celltype = {}

    fig, axs = plt.subplots(ncols=5, nrows=len(pdata.obs[pseudobulk_level].unique())//5+1, figsize=(14, 8))
    axs = axs.flatten()
    for i, cell_type in enumerate(pdata.obs[pseudobulk_level].unique()):
        summed_counts_per_celltype[cell_type] = pdata[pdata.obs[pseudobulk_level]==cell_type].copy()
        
        # gene QC visualization

        dc.plot_filter_by_expr(summed_counts_per_celltype[cell_type], 
                               group='pathology.group', 
                               min_count=min_count,
                               min_total_count=min_total_count, 
                               large_n=large_n, 
                               min_prop=min_prop, 
                               ax=axs[i],
                               )
        
        axs[i].set_title(cell_type)

        
        expressed_genes_per_celltype[cell_type] = dc.filter_by_expr(summed_counts_per_celltype[cell_type],
                                                                    group='pathology.group', 
                                                                    min_count=min_count,
                                                                    min_total_count=min_total_count, 
                                                                    large_n=large_n, 
                                                                    min_prop=min_prop)
    plt.show()

In [11]:
# convert nested list of Seurat object into Rpy2 object 

if count_agg_strategy == 'standardbulk':
    summed_counts_per_celltype =  robjects.ListVector( {
                                                        cell_type: summed_counts_per_celltype[cell_type]
                                                    
                                                        for cell_type in summed_counts_per_celltype.keys()
                                                        }
                                                    )

    expressed_genes_per_celltype =  robjects.ListVector( {
                                                        cell_type: expressed_genes_per_celltype[cell_type]
                                                    
                                                        for cell_type in expressed_genes_per_celltype.keys()
                                                        }
                                                    )
else:
    summed_counts_per_celltype = 'NULL'
    expressed_genes_per_celltype = 'NULL'

In [12]:
%%R -i summed_counts_per_celltype -i expressed_genes_per_celltype

if (summed_counts_per_celltype == 'NULL'){
    print('Standardbulk method not implemented.')
} else {
    print('done')
}

[1] "Standardbulk method not implemented."


## **Pseuobulking With Strategy From [Joel W. Blanchard et al. Nature. 2022](https://doi.org/10.1038/s41586-022-05439-w)**

In [13]:
%%R

if (count_agg_strategy == 'blanchardbulk'){
    
    conflicts_prefer(base::unname)
    
    source('../scripts/functions/qc_and_annotation_aux_functions.r')

    sce <- adata_annot

    print('normalizing counts')
    sce = normalize.default(sce)

    # standardize subject IDs
    meta = colData(sce)
    meta[, subject_id] <- gsub("\\.", "_", meta[, subject_id])
    cell_labels = rownames(meta)

    # use matrix multiplication to summarize (sum) across counts per cell (including all individuals)
    print('sum counts by individual')
    labels = as.data.frame(as.character(interaction(meta[, pseudobulk_level], meta[, subject_id])))
    summed_logcounts_cellxind = sum_counts(logcounts(sce), labels, cell_labels)
    
    # get averages corresponding to both count matrices
    avs_logcounts_cellxind = t(apply(summed_logcounts_cellxind$summed_counts, 1, function(x){x/summed_logcounts_cellxind$ncells}))

    # split the averages by celltype
    x = (strsplit(colnames(avs_logcounts_cellxind), '[.]'))
    celltype = unlist(lapply(1:length(x), function(i) x[[i]][[1]]))
    individual = unlist(lapply(1:length(x), function(i) x[[i]][[2]]))
    celltype_unique = unique(celltype)

    avs_by_ind_out = list()
    summed_counts_per_celltype = list()
    expressed_genes_per_celltype = list()

    for(i in celltype_unique){

        index = celltype==i
        df = avs_logcounts_cellxind[, index]
        colnames(df) = individual[index]
        avs_by_ind_out[[i]] = df

        # create SingleCellExperiment object for summed counts
        summed_counts_per_celltype[[i]] = SingleCellExperiment(assays = list(logcounts = as.matrix(df)))
        rownames(colData(summed_counts_per_celltype[[i]])) = colnames(df)
        
        metadata <- meta[!duplicated(meta[, subject_id]), ]
        colData(summed_counts_per_celltype[[i]]) <- merge(colData(summed_counts_per_celltype[[i]]), 
                                                    metadata, 
                                                    by = NULL,
                                                    by.x = "row.names",
                                                    by.y = subject_id, 
                                                    all.x = TRUE)

        rownames(rowData(summed_counts_per_celltype[[i]])) <- rownames(df)  

        colData(summed_counts_per_celltype[[i]])[, subject_id] = colData(summed_counts_per_celltype[[i]])$Row.names
        
        # in how many cells per celltype is each gene detected?
        print('getting nonzero counts')

        mask <- colData(sce)[, pseudobulk_level] == i
        sce_temp <- sce[, mask]
        meta_temp <- as.data.frame(colData(sce_temp)[, pseudobulk_level])

        counts_nonzero = counts(sce_temp)>0
        
        # determine expressed genes based on threshold
        detected_genes_cell = sum_counts(counts_nonzero, label = meta_temp, cell_labels)
        fraction_detected_genes_cell = t(apply(detected_genes_cell$summed_counts, 1, function(x){x/detected_genes_cell$ncells}))

        # get expression list
        fraction_detected_genes_cell_binary = fraction_detected_genes_cell>gene_celltype_threshold
        genes = rownames(fraction_detected_genes_cell)
        expressed = lapply(colnames(fraction_detected_genes_cell_binary), function(x) genes[unname(fraction_detected_genes_cell_binary[,x])])
        names(expressed) = colnames(fraction_detected_genes_cell_binary)

        expressed_genes_per_celltype[[i]] <- names(expressed)
            
    }

} else {
  summed_counts_per_celltype <- NULL
  names(summed_counts_per_celltype) <- NULL

  print('Blanchardbulk method not implemented.')
}

[1] "Blanchardbulk method not implemented."


## **Pseudocell approach [Gazestani. et. al. 2023](https://www.sciencedirect.com/science/article/pii/S0092867423008590?via%3Dihub)**

### **Pseudocell construction** 

On one hand, cell-based methods for differential expression are time-consuming and sensitive to single-cell data dropouts. On the other hand, pseudobulk methods, which aggregate cell expressions into a single sample (as with the `standardbulk` approach above), address these issues but may lead to a loss of statistical power and neglect variations in confidence levels between individuals with different cell-type sampling sizes.

To address these challenges pseudocell/metacell methods have been introduced this method lies between the cell-based and pseudobulk-based approaches, so it can take advantage of each method while remedying the issues attributed to them.

According to **[Gazestani. et. al. 2023](https://www.sciencedirect.com/science/article/pii/S0092867423008590?via%3Dihub)**, `we first aggregate the raw UMI count` of, on average, every 30 cells per subject and cell type. 

We construct one pseudocell for cell types that have between `15 to 45 cells in a donor` and exclude cell types that had less than `15 cells`. This pseudocell-based analysis reduces the impact of dropout and technical variability, while ameliorating low statistical power and high variation in sample size issues attributed to the pseudobulk approaches. 

**[Gazestani. et. al. 2023](https://www.sciencedirect.com/science/article/pii/S0092867423008590?via%3Dihub)**, provide functions:

- `To generate pseudocells that are composed of, on average, 20 cells or higher. This function can also generate pseudobulk samples.`

- `To generate pseudocells that are on average composed of 10 cells`

### **Available Aggregration Methods**


1. #### **Pseudocells from low-dim network embedding**

The main arguments for running the function:

- `parsing.col.names`: a vector of column names to be used to parse the data. Usually the columns related to the donor id and the cell type annotation (see above figure).

- `pseudocell.size`: the average size of pseudcoells. If NULL, generates pseudobulk data.

- `inputExpData`: Input single cell data object in format.

- `min_size_limit`: minimum acceptable size of the pseudocells. usually 10 or 15.

- `inputPhenoData`: the meta data on the cells that matches in order with the input PC space to use to create the similarity network. It's highly recommended to be used if. defualt: NULL.

- `nPCs`: Number of pcs/dimensions to use to create the similarity network. Used only if human or mouse

- `rand_pseudobulk_mod`: To randomly assign cells to pseudocells in each cell type from each individual; or create a similarity network between these cells and use that similarity net to group cells with highly similar expression patterns to each other. `default: TRUE, but Here we set to FALSE``

The authors reiterate that it's highly recommended to provide if the function will generate the embedding for each cell type in each individual. it's possible that such embedding is driven by the quality of the cells in the group as opposed to biological variation, and hence quality of the pseudocells and inference on them would be limited.


2. #### **Pseudocells with random cell assignment**

Alternatively, we can create pseudocells using a method that randomly assign cells to pseudocells in each cell type from each individual by setting `rand_pseudobulk_mod=TRUE`. Rather than the previous method which creates a similarity network between these cells and uses that similarity network to group cells with highly similar expression patterns to each other.


3. #### **Pseudocells with smaller size**

We can also generate pseudocells using that allows us to have pseudocells of smaller size. The main arguments for this function:

- `inputExpData`: Input single cell data object in SingleCellExperiment format

- `embeddings`: Input PC space to use to create the similarity network. It can't be set to NULL

- `pseudobulk_split_col`: The column name to parse the single cell data.

- `min_dataset_size`: Minimum number of acceptable pseudocell size. default: 4.


In [14]:
############## PARAMETERS FOR NETWORK AND RANDOM #####################

pseudocell_size = 30    # the average size of pseudcoells. If NULL, generates pseudobulk data.
nDims = 20              # Number of dimensions to use to create the similarity network. 
organism = 'Human'      # Used only if human or mouse
nCores = 40 

############### PARAMETERS FOR 'SMALL NETWORK' #####################

pseudobulk_split_col = "lib_anno"
min_dataset_size = 4 

In [15]:
%%R -i organism -i nDims -i pseudocell_size -i pseudobulk_split_col -i min_dataset_size -i nCores

library(scuttle)
library(Matrix)
library(ensembldb)
library(EnsDb.Hsapiens.v86)

conflicts_prefer(GSEABase::setdiff)
conflicts_prefer(SingleCellExperiment::counts)

source("../scripts/functions/sconline_code.R")

# generating the embedding space
if (count_agg_strategy %in% c('network', 'smaller_network', 'random')){

  if (as.logical(reconstruct_embedding)){
    exp_seurat = .extraExport2SeuratFn(adata_annot) %>%
                  Seurat::NormalizeData() %>%
                      FindVariableFeatures() %>% 
                      ScaleData() %>% 
                      RunPCA(verbose=F)

    embedding_data = exp_seurat@reductions$pca@cell.embeddings[,1:nDims]
  } else{
    embedding_data = reducedDim(adata_annot, embedding_name)
  }

}


if (count_agg_strategy == 'network'){

  # generating the pseudocells
    
  # Function adds/modifies three annotation columns: pseuodcell_size, QC_Gene_total_count, QC_Gene_unique_count
  # QC_Gene_total_count: equivalant to nUMI for the pseudobulk samples
  # QC_Gene_unique_count: equivalant to nGene for the pseudobulk samples
  # use scale(tst$QC_Gene_total_count) and scale(tst$pseudocell_size) as additional covariates for the DE analysis

  res = suppressWarnings(.sconline.PseudobulkGeneration(argList = NULL, 
                                  # The columns in the pheonData that will be used to parse the expression data 
                                  # and generate the pseudocell/pseudobulk data
                                    parsing.col.names = c(subject_id, pseudobulk_level), 
                                  # average pseudocell size.
                                    pseudocell.size = pseudocell_size,
                                    inputExpData = adata_annot,
                                  # minimum acceptable size (ie, #cells) for each pseudobulk
                                    min_size_limit = min_size_limit,
                                  # in case we want to run the function outside sconline space
                                    inputPhenoData = as.data.frame(colData(adata_annot)),
                                  # the embedding space to be used for the generation of the pseudobulk.
                                  # only needed when pseudocell.size is not null
                                    inputEmbedding = embedding_data, 
                                  # the dimension of the embedding space for the construction of pseudobulk data
                                    nPCs = nDims, 
                                    ncores = nCores,
                                    rand_pseudobulk_mod = F,
                                  # used to identify and estimate Mitochondrial Genes
                                    organism = organism))
                                    
} else  if (count_agg_strategy == 'random'){

  res = suppressWarnings(.sconline.PseudobulkGeneration(argList=NULL, 
                                  # The columns in the pheonData that will be used to parse the expression data 
                                  # and generate the pseudocell/pseudobulk data
                                    parsing.col.names = c(subject_id, pseudobulk_level), 
                                  # average pseudocell size.
                                    pseudocell.size = pseudocell_size,
                                    inputExpData = adata_annot,
                                  # minimum acceptable size (ie, #cells) for each pseudobul
                                    min_size_limit = min_size_limit,
                                  # in case we want to run the function outside sconline space
                                    inputPhenoData = as.data.frame(colData(adata_annot)),
                                    ncores = nCores,
                                    rand_pseudobulk_mod = T,
                                  # used to identify and estimate Mitochondrial Genes
                                    organism = organism))   
                                
}  else if (count_agg_strategy == 'pseudobulk'){

  res = suppressWarnings(.sconline.PseudobulkGeneration(argList = NULL, 
                                  # The columns in the pheonData that will be used to parse the expression data 
                                  # and generate the pseudocell/pseudobulk data
                                    parsing.col.names = c(subject_id, pseudobulk_level),  
                                  # average pseudocell size.
                                    pseudocell.size = NULL,
                                    inputExpData = adata_annot,
                                  # minimum acceptable size (ie, #cells) for each pseudobulk
                                    min_size_limit = min_size_limit,
                                  # in case we want to run the function outside sconline space
                                    inputPhenoData = as.data.frame(colData(adata_annot)),
                                  # the embedding space to be used for the generation of the pseudobulk.
                                  # only needed when pseudocell.size is not null
                                    inputEmbedding = NULL, 
                                  # the dimension of the embedding space for the construction of pseudobulk data
                                    nPCs = NULL, 
                                    ncores = nCores,
                                    rand_pseudobulk_mod = F,
                                    organism = organism))

} else  if (count_agg_strategy == 'smaller_network') {   

  adata_annot$lib_anno = paste0(adata_annot[[subject_id]], "_", adata_annot[[pseudobulk_level]])

  # creates pseudobulk samples of median size 10
  res = .sconline.Pseudobulk10(inputExpData=adata_annot,
                              embeddings = embedding_data,
                              pseudobulk_split_col = "lib_anno",
                              min_dataset_size = min_dataset_siz,
                              organism = organism)   

}

if (count_agg_strategy %in% c('network', 'smaller_network', 'random', 'pseudobulk')){
  res$QC_Gene_total_log=log2(res$QC_Gene_total_count)
  res$QC_Gene_unique_log=log2(res$QC_Gene_unique_count)
  res$QC_MT_log=log2(res$QC_MT.pct)
  
  summed_counts_per_celltype <- lapply(unique(colData(res)[, pseudobulk_level]), function(cell_type) res[, colData(res)[, pseudobulk_level] == cell_type,])
  names(summed_counts_per_celltype) <- unique(colData(res)[, pseudobulk_level]) 
}

[conflicted] Will prefer GSEABase::setdiff over any other package.
[conflicted] Will prefer SingleCellExperiment::counts over any other package.
Main Functions:
.myRead10X()
.myRead10X_h5()
.myLigerToExpSet()
.mycBindFn()
.myExpSetCreatorFn()
.myIntegrative_oneline()
.myFindAllMarkers()
.myAnnotateFn()
.my2dPlot()
.myPseudoCellfn_v2()
.myLabelTransfer_harmony()
.myLabelTransfer_liger()
.myMapToHuman()
.myRiverPlotFn()
.myClusteringOptimizerFn()
.myMarkerBasedAnalysisFn()
.mycellAssignHeatmap()
.myMetaMarkerFn()
.myFindNeighbors()
.myVlnPlot()
.myFeaturePlot()
.myheatmap.3()
.myEvalMarkers()
.myReadGMT()
.mySplitObject()
.myRTNgsea(two-sided GSEA implementation)


Loading required package: GenomicFeatures
Loading required package: AnnotationFilter
qs 0.27.2. Announcement: https://github.com/qsbase/qs/issues/103


## **Metacell approach from [Morabito et al. Cell Reports Methods. 2023](https://www.sciencedirect.com/science/article/pii/S2667237523001273?via%3Dihub)**

We implemented the metacell aggregation approach from the hdWGCNA framework, originally developed to address the inherent sparsity of single-cell expression data for co-expression network construction. The method employs a bootstrapped aggregation (bagging) strategy that preserves biological context by constraining cell groupings within matched sample and cell type boundaries. The method first projects cells into a reduced dimensional space where k-nearest neighbors (KNN) analysis identifies transcriptionally similar cells. Then, using a bootstrapped sampling procedure, cells are iteratively selected and aggregated with their `k nearest neighbors`, where `k ≥ 2`. To prevent redundancy in the final metacell expression matrix, the method uses an overlap threshold (set as `max_shared`) that restricts the selection of cells whose neighbor sets substantially overlap with previously selected cells. The procedure continues until either reaching a target number of metacells or exhausting the pool of valid cells.

This approach is governed by three key parameters: 

- `n_cells_per_metacell`: determines the number of cells per metacell

- `max_shared`: controls the permitted overlap between metacells, and 

- `min_cells` establishes a minimum cell count threshold for group inclusion.

- `gene_selection`: Required only if `pseudobulking_strategy=='metacell'`, indicating the gene selection method for setting up a Seurat object for WGCNA.

- `geneSet`: Required only if `pseudobulking_strategy=='metacell'` and `gene_selction='custom'`, specifying a custom gene set.

The resulting metacell expression profiles demonstrate significantly reduced sparsity while maintaining the underlying biological structure and ameliorating technical artifacts such as dropout events. Gene selection for metacell construction can be specified through either preset criteria or custom gene sets, which enables a flexible adaptation to diverse analytical contexts. The robustness of the derived metacell profiles makes them particularly suitable for downstream applications including correlation analyses, network construction, and pathway activity quantification.

In [16]:
n_cells_per_metacell = 30           # determines the number of cells per metacell
max_shared = 10                     # controls the permitted overlap between metacells, and 
min_cells = 100                     # establishes a minimum cell count threshold for group inclusion. (should be greater than n_cells_per_metacells)
gene_selection = 'fraction'         # Only required if `pseudobulking_strategy=='metacell'``. specifies the gene selection method when setting up seurat object for WGCNA. 
gene_set = ''                       # Only required if `pseudobulking_strategy=='metacell'`` and `gene_selction='custom'`

In [17]:
%%R -i gene_set -i gene_selection -i n_cells_per_metacell -i max_shared -i min_cells

if (count_agg_strategy == 'metacell'){

  seurat_obj <- as.Seurat(adata_annot, counts = "counts", data = "counts")
  
  # Perform dimensionality reduction and plot
  if (as.logical(reconstruct_embedding)){
    
    print('Reconstructing cell embeddings')
    seurat_obj <- FindVariableFeatures(seurat_obj)
    seurat_obj <- ScaleData(seurat_obj)
    seurat_obj <- RunPCA(seurat_obj)
    seurat_obj <- RunHarmony(seurat_obj, group.by.vars = subject_id)
    seurat_obj <- RunUMAP(seurat_obj, reduction='harmony', n.neighbors=15, dims=1:30, min.dist=0.1)
    reduction = 'harmony'
    p <- DimPlot(seurat_obj, group.by = pseudobulk_level, label = TRUE) +
        umap_theme() + ggtitle(subject_id) + NoLegend()

    fig_dir = paste0(save_dir, "/plots/UMAP/")

    if (!dir.exists(fig_dir)) {
      dir.create(fig_dir, recursive=TRUE)
    }

    # Save plot to PDF
    pdf(file = paste0(fig_dir, save_prefix, "_integrated_umap.pdf"), width = 4, height = 4, useDingbats = FALSE)
    print(p)
    dev.off()
  } else {
    print(paste0('Fetching embeddings from ', embedding_name))
    reduction = embedding_name
  }

  print('Setting up data for metacell construction')

  if (gene_selection == 'custom') {
    seurat_obj <- SetupForWGCNA(
      seurat_obj,
      gene_select = "custom",            # the gene selection approach
      gene_list = as.vector(gene_set),   # list of genes to be included
      group.by = pseudobulk_level,       # grouping parameter
      wgcna_name = 'pseudo_cell'         # the name of the hdWGCNA experiment
    )
  } else {
    seurat_obj <- SetupForWGCNA(
      seurat_obj,
      gene_select = "fraction",           # the gene selection approach
      fraction = gene_celltype_threshold, # fraction of cells for gene inclusion
      group.by = pseudobulk_level,        # grouping parameter
      wgcna_name = 'pseudo_cell'          # the name of the hdWGCNA experiment
    )
  }

  print(paste0(length(GetWGCNAGenes(seurat_obj)), " WGCNA Genes"))

  print('Constructing MetaCells')
  
  seurat_obj <- MetacellsByGroups(
    seurat_obj = seurat_obj,
    group.by = c(pseudobulk_level, subject_id),   # specify the columns in seurat_obj@meta.data to group by
    reduction = reduction,                        # select the dimensionality reduction to perform KNN on
    k = n_cells_per_metacell ,                    # nearest-neighbors parameter
    max_shared = max_shared,                      # maximum number of shared cells between two metacells
    min_cells = min_cells,                        # the minimum number of cells in a particular grouping to construct metacells
    ident.group = pseudobulk_level,               # set the Idents of the metacell seurat object
    wgcna_name = 'pseudo_cell',                   # the name of the hdWGCNA experiment
    mode = "sum"                                  # determines how to make gene expression profiles for metacells from their constituent single cells. 
                                                  # Options are "average" or "sum".
    )

  # perform logCPM normalization of metacell expression matrix.
  # print('Normalizing (log Counts Per Million Normalization) of MetaCells')
  # seurat_obj <- NormalizeMetacells(
  #   seurat_obj, 
  #   wgcna_name = 'pseudo_cell',
  #   normalization.method = "LogNormalize",
  #   scale.factor = 1e6)

  ############ Merge Metadata from raw adata to metacells #################
  sce <- as.SingleCellExperiment(GetMetacellObject(seurat_obj))

  metadata_cols <- setdiff(colnames(colData(adata_annot)), pseudobulk_level)
  metadata_raw <- colData(adata_annot)[, metadata_cols]

  metadata_raw <- as.data.frame(metadata_raw)

  metadata_raw <- metadata_raw[!duplicated(metadata_raw[, subject_id]), ]

  metacell_coldata <- as.data.frame(colData(sce))
  merged_coldata <- merge(metacell_coldata, 
                        metadata_raw, 
                        by.x = subject_id, 
                        by.y = subject_id, 
                        all.x = TRUE)

  rownames(merged_coldata) <- rownames(colData(sce))
  colData(sce) <- DataFrame(merged_coldata)

  ####### Split SCE object into named list by cell type ###############
  cell_types <- unique(colData(sce)[[pseudobulk_level]])
  summed_counts_per_celltype <- list()
  
  for (cell_type in cell_types) {
    cells_idx <- colData(sce)[[pseudobulk_level]] == cell_type
    sce_subset <- sce[, cells_idx]
    summed_counts_per_celltype[[cell_type]] <- sce_subset
  }

}

[1] "Fetching embeddings from X_scVI"
[1] "Setting up data for metacell construction"
[1] "11928 WGCNA Genes"
[1] "Constructing MetaCells"


In addition: Warning messages:
1: Keys should be one or more alphanumeric characters followed by an underscore, setting key from X_scVI_ to XscVI_ 
2: Layer 'data' is empty 
3: Layer 'scale.data' is empty 


## **Gene Filtering and Log Normalization**

Gene set variation analysis (`GSVA`) employs a kernel estimator for the cumulative density function, which necessitates input gene expression data in logarithmic scale measurements (`log-CPMs`, `log-RPKMs`, or `log-TPMs`). To prepare the `pseudo-bulked` data for this, we first `filterByExpr` function from `edgeR`. Genes were retained based on two criteria: a minimum count threshold of 0.9 and a minimum total count threshold determined as the maximum of either 2% of the sample size or 10 counts. The filtered expression data was then normalized to account for library size differences and subsequently transformed to log-CPM values using a prior count of 0.25 to manage zero counts. 

The resulting log-transformed values were stored in the `logcounts` assay.

In [18]:
%%R -o expressed_genes_per_celltype -o summed_counts_per_celltype

library(edgeR)

if (count_agg_strategy != 'blanchardbulk'){
    conflicts_prefer(base::intersect)
    
    expressed_genes_per_celltype <- list()

    for (cell_type in names(summed_counts_per_celltype)) {

        inputExpData <- summed_counts_per_celltype[[cell_type]]
        # keep <- filterByExpr(counts(inputExpData),min.count = 0.9, min.total.count = max(0.02*ncol(inputExpData ), 10))

        dge <- DGEList(counts=counts(inputExpData))
        # dge <- dge[keep, ,keep.lib.sizes=FALSE]
        dge <- calcNormFactors(dge)
        
        # Create logCPM object for log-transformed expression values
        logCPM <- new("EList")
        logCPM$E <- edgeR::cpm(dge, normalized.lib.sizes = TRUE, log = TRUE, prior.count = 0.25)
        
        assays(summed_counts_per_celltype[[cell_type]])[["logcounts"]]  <- logCPM$E
        expressed_genes_per_celltype[[cell_type]] <- as.vector(rownames(logCPM$E))
    }
}

[conflicted] Will prefer base::intersect over any other package.


In addition: Warning message:
In asMethod(object) :
  sparse->dense coercion: allocating vector of size 2.5 GiB


### **Save Summed Counts and Expressed Genes**

In [19]:
if cell_group=="all":
    cell_supertypes = list(chain(*list(subclass.values()))) 
else:
    cell_supertypes = [subclass[cell_group][cell_group_number]]

subclass = robjects.ListVector(subclass)

In [20]:
%%R -i data_dir -i subclass -i cell_supertypes

source('../scripts/functions/helper_functions.r')

for (cell_type in names(summed_counts_per_celltype)){
  # Save summed counts
  saveRDS(
    summed_counts_per_celltype[[cell_type]],
    file.path(
      data_dir,
      "counts",
      cell_group,
      clean_strings(cell_supertypes[1], preserve_case = TRUE),
      paste0(clean_strings(cell_type, preserve_case = TRUE), "_", count_agg_strategy,"_count_data.rds")
    )
  )
  
  # Save expressed genes
  saveRDS(
    expressed_genes_per_celltype[[cell_type]],
    file.path(
      data_dir,
      "counts",
      cell_group,
      clean_strings(cell_supertypes[1], preserve_case = TRUE),
      paste0(clean_strings(cell_type, preserve_case = TRUE), "_", count_agg_strategy,"_expressed_genes.rds")
    )
  )
}

In [21]:
%%R 

library(scMerge)
source('../scripts/functions/helper_functions.r')

# Convert assays to matrix format
for(i in seq_along(summed_counts_per_celltype)) {
  # Convert counts
  if("counts" %in% assayNames(summed_counts_per_celltype[[i]])) {
    assay(summed_counts_per_celltype[[i]], "counts") <- 
      as.matrix(assay(summed_counts_per_celltype[[i]], "counts"))
  }
  
  # Convert logcounts
  if("logcounts" %in% assayNames(summed_counts_per_celltype[[i]])) {
    assay(summed_counts_per_celltype[[i]], "logcounts") <- 
      as.matrix(assay(summed_counts_per_celltype[[i]], "logcounts"))
  }
}

# merge supertypes into the main subclass/cell_type and save
all_expressed_genes <- unique(unlist(expressed_genes_per_celltype))
all_summed_counts <- sce_cbind(
  summed_counts_per_celltype,
  method = "union",
  cut_off_batch = 0.00,
  cut_off_overall = 0.00,
  exprs = c("counts", "logcounts"),
  colData_names = colnames(colData(summed_counts_per_celltype[[1]])),
  batch_names = NULL
  )

saveRDS(
  all_summed_counts,
  file.path(
    data_dir,
    "counts",
    cell_group,
    clean_strings(cell_supertypes[1], preserve_case = TRUE, replace_hyphen=FALSE),
    paste0(clean_strings(cell_supertypes[1], preserve_case = TRUE, replace_hyphen=FALSE), "_", count_agg_strategy,"_count_data.rds")
  )
)

saveRDS(
  all_expressed_genes,
  file.path(
    data_dir,
    "counts",
    cell_group,
    clean_strings(cell_supertypes[1], preserve_case = TRUE, replace_hyphen=FALSE),
    paste0(clean_strings(cell_supertypes[1], preserve_case = TRUE, replace_hyphen=FALSE), "_", count_agg_strategy, "_expressed_genes.rds")
  )
)

summed_counts_per_celltype


$Pvalb
class: SingleCellExperiment 
dim: 36601 9175 
metadata(0):
assays(2): counts logcounts
rownames(36601): MIR1302-2HG FAM138A ... AC007325.4 AC007325.2
rowData names(0):
colnames(9175): Pvalb#H19.33.004_1 Pvalb#H19.33.004_2 ...
  Pvalb#H21.33.047_130 Pvalb#H21.33.047_131
colData names(18): donor_id orig.ident ... pmi apoe4_status
reducedDimNames(0):
mainExpName: originalexp
altExpNames(0):



The assay named 'counts' will be used to determine the proportion of zeroes for each batch
In addition: Warning message:
In asMethod(object) :
  sparse->dense coercion: allocating vector of size 2.5 GiB
